In [1]:
# for testing whether CPU or GPU is faster on a problem
NO_GPU = False

if NO_GPU:
    import os
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    
import numpy as np
import matplotlib.pyplot as plt
import keras
import keras.layers as kl

Using TensorFlow backend.


In [2]:
def build_model(*layers, verbose=False,
               optimizer='adam', loss='binary_crossentropy', metrics=['acc'],
               compile_kwargs={}):
    
    model = keras.models.Sequential()
    
    for layer in layers:
        model.add(layer)
        
    if verbose:
        print("Model summary:")
        model.summary()
    
    for kw in ('optimizer', 'loss', 'metrics'):
        if not kw in compile_kwargs:
            compile_kwargs[kw] = locals()[kw]
    model.compile(**compile_kwargs)
    
    return model

In [3]:
def is_sorted(a):
    return np.all(a[:-1] <= a[1:])

def get_sorted(a):
    assert len(a.shape) == 1 # is flat
    return np.sort(a)

def get_unsorted(a):
    assert len(a.shape) == 1 # is flat
    if not is_sorted(a):
        return a
    return get_unsorted(np.random.permutation(a))

def get_sequence_label_pair(sequence_length, from_numbers, sorted_label=1, unsorted_label=0):
    sequence = np.random.choice(from_numbers, size=sequence_length)
    sort = np.random.randint(0, 2) # toss a coin
    if sort:
        return get_sorted(sequence), sorted_label
    else:
        try:
            return get_unsorted(sequence), unsorted_label
        except RecursionError: # happens when all chosen elements are the same
            # let's try again, but not more times, I might have been stupid elsewhere
            return get_unsorted(np.random.choice(from_numbers, size=sequence_length)), unsorted_label

def data_generator(batch_size, data_shape, from_numbers, sorted_label=1, unsorted_label=0):
    sequence_length = np.product(data_shape)
    while True:
        data = np.zeros((batch_size, sequence_length))
        labels = []
        
        for i in range(batch_size):
            x, y = get_sequence_label_pair(sequence_length, from_numbers, sorted_label, unsorted_label)
            data[i] = x
            labels.append(y)

        yield data.reshape((batch_size, *data_shape)), np.asarray(labels)

def variable_length_data_generator(batch_size, sequence_lengths, from_numbers, sorted_label=1, unsorted_label=0):
    while True:
        sequence_length = np.random.choice(sequence_lengths)
        data = np.zeros((batch_size, sequence_length))
        labels = []
        
        for i in range(batch_size):
            x, y = get_sequence_label_pair(sequence_length, from_numbers, sorted_label, unsorted_label)
            data[i] = x
            labels.append(y)

        yield data.reshape((batch_size, sequence_length, 1)), np.asarray(labels)


In [4]:
def plot_history(history, figsize=(15,4), title='', columns=2, start_x_from=0):
    """Graphs a history for each key (combines validation and training keys into one plot).
    
    start_x_from=N skips the first N entries.
    
    History can be a whole training history class or just a dict."""
    
    if hasattr(history, 'history'): # full history given
        history = history.history   # only that history is enough
        
    assert hasattr(history, 'keys')
    keys = [key for key in history.keys() if not key.startswith("val_")]
    assert keys # there is one at least
    epochs = list(range(1,len(history[keys[0]])+1)) # all should have the same size list
    
    rows = np.ceil(len(keys)/columns).astype('int')
    
    fig=plt.figure(figsize=figsize)
    f = plt.title(title)
    f.axes.get_xaxis().set_visible(False)
    f.axes.get_yaxis().set_visible(False)
    
    i = 1
    for key in sorted(keys):
        valkey = "val_" + key
        fig.add_subplot(rows, columns, i)
        i += 1
        plt.plot(epochs[start_x_from:], history[key][start_x_from:], label="Training " + key,
                 marker='.', color='#00A287', linestyle='')
        
        late_avg = np.mean(history[key][(len(history[key]) * 90) // 100 : ])
        plt.plot((epochs[start_x_from], epochs[-1]), (late_avg, late_avg),
                 color="#74E600", label='Mean {:.3f}'.format(late_avg))
        if valkey in history:
            plt.plot(epochs[start_x_from:], history[valkey][start_x_from:], label='Validation ' + key,
                    marker='+', color='#DF004F', linestyle='')
            
            late_avg = np.mean(history[valkey][(len(history[valkey]) * 90) // 100 : ])
            plt.plot((epochs[start_x_from], epochs[-1]), (late_avg, late_avg),
                     color="#FF6700", label='Mean {:.3f}'.format(late_avg))
        plt.legend()
        
    plt.show()

In [5]:
SEQ_LENGTH = 10;

model = build_model(
    kl.InputLayer((SEQ_LENGTH, 1)),
    kl.Conv1D(1, 2, activation='relu', use_bias=False),
    kl.Flatten(),
    kl.Dense(1, activation='sigmoid', use_bias=False),
    loss='binary_crossentropy',
    verbose=0
)

w = model.get_weights()

w[0] = np.array([1.0, -1.0], dtype='float32').reshape(w[0].shape)
w[1] = np.ones_like(w[1])

model.set_weights(w)

gen = data_generator(
    batch_size=100,
    data_shape=(SEQ_LENGTH, 1),
    from_numbers=10*np.random.sample(10000)-5,
    sorted_label=0,
    unsorted_label=1
)

print("{0[0]}: {1[0]}\n{0[1]}: {1[1]}".format(model.metrics_names, model.evaluate_generator(gen, steps=500)))

Instructions for updating:
Colocations handled automatically by placer.
loss: 0.3450459977388382
acc: 1.0


In [6]:
SEQ_LENGTH = 10;
# best so far:
# relu, softsign
# elu, sigmoid
# elu, softsign - not working (90% and then decreases)
model = build_model(
    kl.InputLayer((SEQ_LENGTH, 1)),
    kl.Conv1D(1, 2, activation='elu', use_bias=True),
    kl.Flatten(),
    kl.Dense(1, activation='sigmoid', use_bias=True),
    loss='binary_crossentropy',
    verbose=0
)

gen = data_generator(
    batch_size=100,
    data_shape=(SEQ_LENGTH, 1),
    from_numbers=4*np.random.sample(10000)-2,
    sorted_label=0,
    unsorted_label=1
)

In [7]:
SEQ_LENGTH = 10;
# best so far:
# relu, softsign
# elu, sigmoid (sometimes gets stuck, better to use more conv units, reaches 100%)
# elu, softsign - not working (90% and then decreases)
model = build_model(
    kl.InputLayer((SEQ_LENGTH, 1)),
    kl.Conv1D(1, 2, activation='elu', use_bias=True),
    kl.Flatten(),
    kl.Dense(1, activation='sigmoid', use_bias=True),
    loss='binary_crossentropy',
    verbose=0
)

gen = data_generator(
    batch_size=100,
    data_shape=(SEQ_LENGTH, 1),
    from_numbers=0.01*(np.random.sample(10000)-0.5),
    sorted_label=0,
    unsorted_label=1
)

model.fit_generator(gen, steps_per_epoch=1000, epochs=200)

print("{0[0]}: {1[0]}\n{0[1]}: {1[1]}".format(model.metrics_names, model.evaluate_generator(gen, steps=500)))

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6874 - acc: 0.7142
Epoch 2/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6525 - acc: 0.9195
Epoch 3/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6002 - acc: 0.9348
Epoch 4/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.5427 - acc: 0.9422
Epoch 5/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.4862 - acc: 0.9445
Epoch 6/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.4334 - acc: 0.9460
Epoch 7/200
 692/1000 [===================>..........] - ETA: 0s - loss: 0.3923 - acc: 0.9471

KeyboardInterrupt: 

In [ ]:
model = build_model(
    kl.InputLayer((None, 1)),
    kl.GRU(1),
    kl.Dense(1, activation='sigmoid', use_bias=True),
    loss=keras.losses.hinge,
    verbose=1
)

In [ ]:
gen = variable_length_data_generator(
    batch_size=100,
    sequence_lengths=(5,10,10),
    from_numbers=4*(np.random.sample(10000)-0.5),
    sorted_label=1,
    unsorted_label=0
)

model.fit_generator(gen, steps_per_epoch=1000, epochs=10)

print("{0[0]}: {1[0]}\n{0[1]}: {1[1]}".format(model.metrics_names, model.evaluate_generator(gen, steps=50)))

In [ ]:
model = build_model(
    kl.InputLayer((None, 1)),
    kl.GRU(2),
    kl.Dense(2, activation='elu', use_bias=True),
    kl.Dense(2, activation='softmax', use_bias=True),
    loss=keras.losses.categorical_crossentropy,
    verbose=1
)

In [ ]:
gen = variable_length_data_generator(
    batch_size=100,
    sequence_lengths=(2000),
    from_numbers=4*(np.random.sample(10000)-0.5),
    sorted_label=[1,0],
    unsorted_label=[0,1]
)

#model.fit_generator(gen, steps_per_epoch=1000, epochs=10)

print("{0[0]}: {1[0]}\n{0[1]}: {1[1]}".format(model.metrics_names, model.evaluate_generator(gen, steps=50)))

In [ ]:
model = build_model(
    kl.InputLayer((None, 1)),
    kl.GRU(2),
    #kl.Dense(2, activation='elu', use_bias=True),
    kl.Dense(1, activation='sigmoid', use_bias=True),
    loss=keras.losses.binary_crossentropy,
    verbose=1
)

In [ ]:
gen = variable_length_data_generator(
    batch_size=1,
    sequence_lengths=(20000,),#(2,2,4,5,10,50),
    from_numbers=4*(np.random.sample(10000)-0.5),
    sorted_label=1,
    unsorted_label=0
)

#model.fit_generator(gen, steps_per_epoch=1000, epochs=10)

print("{0[0]}: {1[0]}\n{0[1]}: {1[1]}".format(model.metrics_names, model.evaluate_generator(gen, steps=50)))

In [ ]:
def greeting(name: str, times: int) -> str:
    return int(times)*'Hello ' + name


In [ ]:
model = build_model(
    kl.InputLayer((None, 1)),
    kl.Conv1D(6, 2, use_bias=False, activation='elu'),
    kl.Conv1D(1,1, activation='elu'),
    kl.GlobalMaxPool1D(),
    kl.Dense(1, activation='sigmoid', use_bias=True),
    # not so good: cosine, cos_prox
    # good: MSLE, KLD, MAE, MAPE, MSE, absolute_import, bin_crossentropy,
    # cat_hinge
    loss=keras.losses.deserialize,
    verbose=1
)

In [ ]:
gen = variable_length_data_generator(
    batch_size=100,
    sequence_lengths=(2,2,4,5,10,50),
    from_numbers=2*(np.random.sample(10000)-0.5),
    sorted_label=0,
    unsorted_label=1
)

model.fit_generator(gen, steps_per_epoch=1000, epochs=100)

print("{0[0]}: {1[0]}\n{0[1]}: {1[1]}".format(model.metrics_names, model.evaluate_generator(gen, steps=50)))

In [ ]:
gen = variable_length_data_generator(
    batch_size=100,
    sequence_lengths=(2,2,4,5,10,50),
    from_numbers=range(4, 65),
    sorted_label=0,
    unsorted_label=1
)

#model.fit_generator(gen, steps_per_epoch=1000, epochs=10)

print("{0[0]}: {1[0]}\n{0[1]}: {1[1]}".format(model.metrics_names, model.evaluate_generator(gen, steps=50)))

In [ ]:
losses = {v for k, v in keras.losses.__dict__.items() if callable(v)}

loss_ftions = [l for l in losses if input(str(l)) != 'n']

In [ ]:
def train_and_show_hist(lossf, model_layers):
    model = build_model(*model_layers, loss=lossf, verbose=0)
    
    hist = model.fit_generator(gen, steps_per_epoch=1000, epochs=15, verbose=1)
    
    plot_history(hist, (16,4), title=str(lossf), start_x_from=3)
    

In [ ]:
models = [
    (
        kl.InputLayer((None, 1)),
        kl.Conv1D(6, 2, use_bias=False, activation='elu'),
        kl.Conv1D(1,1, activation='elu'),
        kl.GlobalMaxPool1D(),
        kl.Dense(1, activation='sigmoid', use_bias=True),
    ),
    (
        kl.InputLayer((None, 1)),
        kl.GRU(1),
        kl.Dense(1, activation='sigmoid', use_bias=True)
    ),
    (
        kl.InputLayer((None, 1)),
        kl.GRU(2),
        kl.Dense(2, activation='elu', use_bias=True),
        kl.Dense(2, activation='softmax', use_bias=True)
    )]

In [ ]:
gen = variable_length_data_generator(
    batch_size=100,
    sequence_lengths=(2,2,4,5,10,50),
    from_numbers=2*(np.random.sample(10000)-0.5),
    sorted_label=0,
    unsorted_label=1
)

for model_layers in models:
    for v in loss_ftions:
        if callable(v):
            for i in range(4):
                try:
                    train_and_show_hist(v, model_layers)
                except Exception as err:
                    print(err)

In [ ]:
for i in range(100):
    x, y = get_sequence_label_pair(L, 10*np.random.sample(100)-5)
    
    y = -1*y + 1
    
    #print(np.max(np.concatenate(((-x[1:] + x[:-1]), np.zeros(9))).reshape(2, 9), axis=0))
    print(x, y, model.predict(x.reshape(1, 10, 1)).flatten()[0])